# Project - Apache Spark & Elastichsearch

##### Students:
* Lilia IZRI      (DS)
* Yacine MOKHTARI (DS)
* Alexandre COMBEAU (DS)

##### Report
[PENSER A METTRE UN LIEN ICI]


In [2]:
pip install tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 KB 1.2 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Imports
import tweepy
from tweepy.auth import OAuthHandler
from tweepy import Stream
import socket
import json

## I. Processing Twitter as a Stream

### 1. Settings : Insert Twitter developer credentials

In [4]:
# KEYS (Remarque mdr c'est des clés que j'ai trouvées sur un site 🤷🏻‍♀️)
consumer_key    = '3XwgFgsXucaFXOrkZtlwvxV5O'
consumer_secret = 'LNnTcxibKalylVImljDsKfkRqb6WHD8I7hdSYt7Pm7VaCetqZm'
access_token    = '608048932-AM21gcwLORlm5j0514DwPL850byLb7Gs04bPZ6Mn'
access_secret   = 'QtnJ8mV75peV7L7Kd4DZKogMRIxzjk5XFOPtZiz62NoM3'

### 2. Creating a new listener class

In [5]:
# Class that represents our Listener
class TweetsListener(Stream):

    def __init__(self, *args, csocket):
        super().__init__(*args)
        self.client_socket = csocket
    def on_data(self, data):
        try:
            # Extracting interesting fields
            tweet = json.loads( data )
            text = tweet['text']
            user = tweet['user']['screen_name']
            location = tweet['user']['location'] if tweet['user']['location'] is not None else 'Unknown'
            date = tweet['created_at']
            
            # Handle hashtags (we join the list by adding '#' and separate them by a space ' '
            list_hashtags = tweet['entities']['hashtags']
            hashtags = " ".join(['#' + i['text' ] for i in list_hashtags]) if list_hashtags is not [] else ""
            
            
            # We combine data and metadata to send them 
            #  We add a key ###field:### so we can split the fields easily 
            #  We remove '\n' from a tweet and put one '\n' between tweets 
            tweet_info = (" ###:field:### user: " + user + " ###:field:### tweet: " + text +\
                          " ###:field:### date: " + date + " ###:field:### location: " + location + \
                          " ###:field:### hashtags: " + hashtags 
                          ).replace('\n', ' ') + '\n'
            
            print(tweet['user']['location'])
            print(tweet_info)
            
            # Send to socket
            self.client_socket.send(tweet_info.encode('utf-8'))
            return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

    # Handle errors
    def if_error(self, status):
        print(status)
        return True

###  3. Authenticate our connection using the pre-defined credentials
In order to begin streaming, we have to make sure to authenticate this connection

In [6]:
def send_tweets(c_socket):
    """
        Send tweets to client socket (in our case ✨SPARK✨)
    """
    twtr_stream = TweetsListener(
        consumer_key, consumer_secret,
        access_token, access_secret,
        csocket=c_socket
    )
    twtr_stream.filter(track=['Depp'])

###  4. Testing connection with client (Spark)
In order to begin streaming, we have to make sure to authenticate this connection

In [7]:
new_skt = socket.socket()         # initiate a socket object
host = "127.0.0.1"                # address host
port = 5552                       # specifie port 
new_skt.bind((host, port))        # Binding host and port

In [ ]:
print(f"Now listening on port: {port}")
new_skt.listen(5)                 # waiting for client connection
c, addr = new_skt.accept()        # Establish connection with client

print(f"Received request from: {addr}")
send_tweets(c)                    # send tweets to client socket

Now listening on port: 5552
Received request from: ('127.0.0.1', 42918)
US
 ###:field:### user: AcidKritana ###:field:### tweet: RT @internationalm: #AmberTurd is a liar: the makeup brand she claimed to use to cover her bruises? Didn't hit the shelves til AFTER her al… ###:field:### date: Sun May 01 19:14:00 +0000 2022 ###:field:### location: US ###:field:### hashtags: #AmberTurd

None
 ###:field:### user: Eza50295396 ###:field:### tweet: RT @lnfosSeriesFR: #PHOTOS Nouveau photoshoot de Johnny Depp pour Dior https://t.co/lUWiAXAaF1 ###:field:### date: Sun May 01 19:14:00 +0000 2022 ###:field:### location: Unknown ###:field:### hashtags: #PHOTOS

Champions du monde
 ###:field:### user: jmikaelson_ ###:field:### tweet: RT @SeriesUpdateFR: Johnny Depp pour Dior.💜 https://t.co/FjasVflKJW ###:field:### date: Sun May 01 19:14:01 +0000 2022 ###:field:### location: Champions du monde ###:field:### hashtags: 

بلد الخواجة في احلامي
 ###:field:### user: mo_o7amed_ ###:field:### tweet: دا افضل مح